In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install yfinance

In [78]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [29]:
stock = pd.read_csv('/content/drive/MyDrive/NH투자증권 공모전/data/NASDAQ_FC_STK_IEM_IFO.csv', encoding = 'cp949')
stock

,isin_cd,tck_iem_cd,fc_sec_krl_nm,fc_sec_eng_nm
0,US00211V1061,AACG,ATA ...,ATA CreatGlo ...
1,US00032Q1040,AADI,Aadi Bioscience ...,Aadi Bioscience ...
2,US02376R1023,AAL,아메리칸 에어라인스 그룹 ...,American Airline ...
3,US03823U1025,AAOI,어플라이드 옵토일렉트로닉스 ...,AOI ...
4,US0003602069,AAON,에이에이온 ...,AAON ...
...,...,...,...,...
2738,US4884452065,ZVRA,Zevra ...,Zevra ...
2739,US98987D1028,ZVSA,Zyversa ...,Zyversa ...
2740,US98985Y1082,ZYME,Zymeworks ...,Zymeworks ...
2741,US98986X1090,ZYNE,자이너바 파마수티컬스 ...,Zynerba Pharms ...


In [37]:
# 'tck_iem_cd' 열의 공백 제거
stock['tck_iem_cd'] = stock['tck_iem_cd'].str.strip()

In [77]:
# 종목 정보 추가 함수
def add_stock_info(df):
    info_columns_mapper = {

        # 정보 관련
        'sector': '섹터',
        'industry': '산업군',
        'longBusinessSummary' : 'description',

        # 매매 정보 관련
        'sharesOutstanding': '발행주식수',
        'averageVolume10days': '종목평균거래량(10일)',
        'averageVolume': '종목평균거래량',
        'heldPercentInstitutions': '기관보유비율',
        'shortRatio': '일일공매도비율',
        'sharesPercentSharesOut': '발행주식대비공매도비율',
        'shortPercentOfFloat': '유동주식중공매도비율',

        # 가격 관련
        'marketCap': '시가총액',  # 200B:mega // 10B~200B:large // 2B-10B:medium // 300M~2B:small // 50M~300M:micro // ~50M:nano
        'currentPrice': '현재가',
        'fiftyDayAverage': '50일평균가',
        'twoHundredDayAverage': '200일평균가',
        'fiftyTwoWeekHigh': '52주최고가',
        'fiftyTwoWeekLow': '52주최저가',
        'SandP52WeekChange': 'S&P_52주변동성',
        '52WeekChange': '52주변동성',
        'ytdReturn': '연초대비수익률',
        'fiveYearAverageReturn': '5년연평균수익률',  # 5년연평균수익률
        'beta': '베타값',  # 5년 데이터, 개별주식의 변동률을 의미. 1에 가까울수록 시장과 가깝고, 1을 넘어가면 시장 대비 고변동, 0으로 가까우면 시장 대비 저변동 주식을 의미함.

        # 현금 창출, 매출 관련 (ttm)
        'totalRevenue': '총매출액',
        'grossProfits': '매출총이익',  # 매출이익(매출액 - 매출원가)
        'revenuePerShare': '주당매출액',
        'ebitda': 'EBITDA',  # 감가상각 등의 부가비용을 차감하기 전의 금액, 영업 활동을 통한 현금 창출 능력. 유형자산의 가치까지 포함하는 지표
        'ebitdaMargins': 'EBITDA마진',  # 유형자산의 유지비용을 고려한 기업의 현금 창출 능력

        # 재무 상태 관련 (mrq)
        'debtToEquity': '부채자본비율',
        'operatingCashflow': '영업현금흐름',  # 영업현금흐름 : 영업이익 - 법인세 - 이자비용 + 감가상각비
        'freeCashflow': '잉여현금흐름',  # 기업의 본원적 영업활동을 위해 현금을 창출하고, 영업자산에 투자하고도 남은 현금
        'totalCashPerShare': '주당현금흐름',
        'currentRatio': '유동비율',  # 회사가 가지고 있는 단기 부채 상환 능력
        'quickRatio': '당좌비율',  # 회사가 가지고 있는 단기 부채 상환 능력
        'overallRisk': '위험 점수',

        # 경영 효율 관련
        'returnOnAssets': '자기자본이익률',  # mrq : 간단히 말해, 얼마를 투자해서 얼마를 벌었냐
        'returnOnEquity': '총자산순이익률',  # mrq : ROE와 비교하여 기업이 가지고 있는 부채의 비중을 볼 때
        'grossMargins': '매출총이익률',  # ttm : 매출이익(매출액 - 매출원가) / 매출액 : 매출이익률, Gross Profit Margin (GPM)
        'operatingMargins': '영업이익률',  # ttm : 매출총이익 - 판관비 - 감가상각비
        'profitMargins': '순이익률',  # ttm : Net Income(순이익) / Revenue(총수익) : 순이익률, Net Profit Margin (NPM)

        # 기업 자산 관련
        'totalCash': '총현금액',
        'totalDebt': '총부채액',

        # 기업 가치 관련
        'priceToBook': 'PBR',  # 기업이 가진 순 자산에 비해 주가가 얼마나 비싼지
        'enterpriseValue': '기업가치',  # 기업가치 : 시가총액 + (총차입금 - 현금성 자산)
        'enterpriseToRevenue': 'EV/R',  # 매출액대비 기업가치 비율
        'enterpriseToEbitda': 'EV/EBITDA',  # EBITDA대비 기업가치 비율 : PER과 의미적으로 비슷한 지표
        'forwardEps': '선행1년EPS',  # 주당순이익, 보통 5년동안의 EPS를 관찰해서 추이를 봄
        'trailingEps': '1년EPS',  # 주당순이익 = 당기순이익 / 유통주식수
        'priceToSalesTrailing12Months': '1년PSR',  # 주가매출액비율 (1년 기준)
        'forwardPE': '선행1년PER',  # 향후 1년동안 예상되는 PER
        'trailingPE': '1년PER',  # 현재 PER. 기업이 한 주당 벌어들이는 순이익에 비해, 실제 주가가 몇 배가 되는 지 나타내는 지표. 고평가 저평가에 사용

        # 성장성 관련
        'revenueGrowth': 'mrq매출액증가율',
        'earningsGrowth': 'mrq수익상승률',
        'earningsQuarterlyGrowth': 'yoy수익상승률',  # yoy : 지난해 동일 분기 대비 최근 분기의 수익 상승률
        'revenueQuarterlyGrowth': 'yoy매출상승률',  # yoy : 지난해 동일 분기 대비 최근 분기의 매출 상승률
        'heldPercentInsiders': '직원보유비율',

        'Research Development': 'R&D비용',
        'Net Income': '순이익',
        'Gross Profit': '매출총이익',
        'Operating Income': '영업이익',
        'Total Revenue': '총매출',
        'Cost Of Revenue': '제품원가',

        'Research Development': 'R&D비용',
        'Net Income': '순이익',
        'Gross Profit': '매출총이익',
        'Operating Income': '영업이익',
        'Total Revenue': '총매출',
        'Cost Of Revenue': '제품원가',

        'Total Liab': '총부채',
        'Total Stockholder Equity': '자기자본',
        'Total Assets': '총자산',
    }

    for i, row in df.iterrows():
        tck_iem_cd = row['tck_iem_cd']  # 종목티커코드 가져오기
        ticker = yf.Ticker(tck_iem_cd)  # 종목티커코드로 Ticker 객체 생성

        # Ticker 객체를 통해 종목 정보 가져오기
        stock_info = ticker.info

        # 필요한 정보만 추출하여 데이터프레임에 추가
        for key, value in info_columns_mapper.items():
            if key in stock_info:
                df.at[i, value] = stock_info[key]

    return df


In [79]:
#데이터프레임 생성
stock_info = add_stock_info(stock)

stock_info

,isin_cd,tck_iem_cd,fc_sec_krl_nm,fc_sec_eng_nm,섹터,산업군,description,발행주식수,종목평균거래량(10일),종목평균거래량,...,mrq매출액증가율,직원보유비율,유동주식중공매도비율,매출총이익,영업현금흐름,잉여현금흐름,위험 점수,1년PER,mrq수익상승률,yoy수익상승률
0,US00211V1061,AACG,ATA ...,ATA CreatGlo ...,Consumer Defensive,Education & Training Services,"ATA Creativity Global, together with its subsi...",31585900.0,3800.0,6822.0,...,0.065,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US00032Q1040,AADI,Aadi Bioscience ...,Aadi Bioscience ...,Healthcare,Biotechnology,"Aadi Bioscience, Inc., a clinical-stage biopha...",24519900.0,130880.0,131912.0,...,0.804,0.12510,0.0387,-1.878100e+07,-5.757300e+07,-3.674612e+07,8.0,NaN,NaN,NaN
2,US02376R1023,AAL,아메리칸 에어라인스 그룹 ...,American Airline ...,Industrials,Airlines,"American Airlines Group Inc., through its subs...",653361984.0,27378180.0,24171950.0,...,0.047,0.01303,0.1125,1.144300e+10,4.345000e+09,1.205875e+09,7.0,3.324742,1.769,1.811
3,US03823U1025,AAOI,어플라이드 옵토일렉트로닉스 ...,AOI ...,Technology,Communication Equipment,"Applied Optoelectronics, Inc. designs, manufac...",33342500.0,2753250.0,4239232.0,...,-0.204,0.08457,0.1936,3.362700e+07,-6.804000e+06,1.881688e+07,NaN,NaN,NaN,NaN
4,US0003602069,AAON,에이에이온 ...,AAON ...,Industrials,Building Products & Equipment,"AAON, Inc., together with its subsidiaries, en...",81609504.0,547790.0,434956.0,...,0.360,0.17443,0.0365,2.375720e+08,1.228380e+08,1.345638e+07,4.0,31.46409,1.734,1.865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,US4884452065,ZVRA,Zevra ...,Zevra ...,Healthcare,Biotechnology,"Zevra Therapeutics, Inc., a rare disease compa...",36207100.0,151370.0,153025.0,...,5.515,0.11387,0.0685,1.011500e+07,-2.380900e+07,-1.657262e+07,NaN,NaN,NaN,NaN
2739,US98987D1028,ZVSA,Zyversa ...,Zyversa ...,Healthcare,Biotechnology,"ZyVersa Therapeutics, Inc., a clinical stage b...",36394200.0,21279420.0,4787927.0,...,NaN,0.54438,0.0110,NaN,-9.196134e+06,-5.211207e+06,NaN,NaN,NaN,NaN
2740,US98985Y1082,ZYME,Zymeworks ...,Zymeworks ...,Healthcare,Biotechnology,"Zymeworks Inc., a clinical-stage biopharmaceut...",67826704.0,300920.0,484875.0,...,0.287,0.04877,0.0866,2.037790e+08,1.647550e+08,8.823938e+07,NaN,2.344322,NaN,NaN
2741,US98986X1090,ZYNE,자이너바 파마수티컬스 ...,Zynerba Pharms ...,Healthcare,Drug Manufacturers—Specialty & Generic,"Zynerba Pharmaceuticals, Inc. operates as a cl...",53939400.0,786110.0,886206.0,...,NaN,0.06070,0.0048,NaN,-3.164196e+07,-1.878967e+07,NaN,NaN,NaN,NaN


In [81]:
stock_info.isnull().sum()

isin_cd             0
tck_iem_cd          0
fc_sec_krl_nm       0
fc_sec_eng_nm       0
섹터                 26
산업군                26
description        25
발행주식수              69
종목평균거래량(10일)        2
종목평균거래량             2
기관보유비율             97
일일공매도비율            26
발행주식대비공매도비율        86
시가총액               63
현재가                18
50일평균가              2
200일평균가             2
52주최고가              0
52주최저가              0
S&P_52주변동성       1661
52주변동성           1661
베타값               221
총매출액              327
주당매출액             329
EBITDA            384
EBITDA마진           25
부채자본비율            555
주당현금흐름             33
유동비율              237
당좌비율              238
자기자본이익률            52
총자산순이익률           222
매출총이익률             25
영업이익률              25
순이익률               25
총현금액               31
총부채액               30
PBR               286
기업가치               28
EV/R              347
EV/EBITDA         386
선행1년EPS           340
1년EPS              30
1년PSR             371
선행1년PER           342
mrq매출액증가율 

In [82]:
stock_info.to_csv('/content/drive/MyDrive/NH투자증권 공모전/data/stock_info.csv', index = False)